In [ ]:
from numpy import *
from random  import *
from copy  import *

#Cardinal directions            
cardinal = {
    'F' : array([ 1, 0, 0], dtype=int32),
    'U' : array([ 0, 1, 0], dtype=int32),
    'R' : array([ 0, 0, 1], dtype=int32),
    'B' : array([-1, 0, 0], dtype=int32),
    'D' : array([ 0,-1, 0], dtype=int32),
    'L' : array([ 0, 0,-1], dtype=int32),
}

def getname(a):
    n=0
    for i in range(3):
        if a[i] == 0:
            n+=1
        else:
            assert(abs(a[i]) == 1)
    assert(n==2)
    
    if a[0] > 0:
        return "+X"
    if a[0] < 0:
        return "-X"
    if a[1] > 0:
        return "+Y"
    if a[1] < 0:
        return "-Y"
    if a[2] > 0:
        return "+Z"
    if a[2] < 0:
        return "-Z"
    
#Make spaces using cross product
def lookat(forward_up_name):
    f = cardinal[forward_up_name[0]]
    u = cardinal[forward_up_name[1]]
    c = cross(f,u)
    M = array([f,u,c],dtype=int32).transpose()
    N=""

    N+=getname(f)
    N+=getname(u)
    N+=getname(c)

    return N, M
    
#combinations - Take every cardinal direction combine with four 90 degree rotations
cmb = []
#forward
cmb += ["FU", "FR", "FD", "FL"]
#backward
cmb += ["BU", "BL", "BD", "BR"]
#Right
cmb += ["RU", "RB", "RD", "RF"]
#Left
cmb += ["LU", "LF", "LD", "LB"]
#Up
cmb += ["UB", "UR", "UF", "UL"]
#Down
cmb += ["DF", "DR", "DB", "DL"]

assert(len(list(set(cmb))) == 24)

O = [lookat(x) for x in cmb]

base = array([cardinal['F'],cardinal['U'],cardinal['R'], array([1,2,3], dtype=int32)])

spaces = []
for n,o in O:
    spaces.append(n)
    if 0:
        print(n)
        print(o)
        print("Base")
        b = base.dot(o)
        print("X:", b[0])
        print("Y:", b[1])
        print("Z:", b[2])
        print("Diag:", b[3])
        print("\n")
print("Spaces:", " ".join(spaces))    
        
#match two arrays of points
#return offset from A2B used if 12 matches found, else return None
def _match(A,B):    
    for a in A:
        for b in B:        
            # compute offset from a to b
            a2b_offset = b-a
            
            #count matches using offset
            m=0
            for aa in A + a2b_offset:
                if (aa==B).all(axis=1).any():
                    m+=1
                    if m > 1:
                        #print("Reached match: ", m, c)
                        pass                    
                    if m >= 12:
                        return a2b_offset
            assert(m>0)
    return None


def match(A,B):    
    for a in A:
        for b in B:        
            # compute offset from a to b
            a2b_offset = b-a            
            C = A + a2b_offset            
            nrows, ncols = C.shape
            dtype={'names':['f{}'.format(i) for i in range(ncols)],'formats':ncols * [C.dtype]}
            m = len(intersect1d(C.view(dtype), B.view(dtype)))
            if m >= 12:
                return a2b_offset
            assert(m>0)
    return None



def computeDistances(points):
    interdistances = []
    l = len(points)
    for i in range(l):
        for j in range(l):
            if  i!= j:
                a = points[i]
                b = points[j]
                c = a-b
                interdistances.append(c[0]*c[0]+c[1]*c[1]+c[2]*c[2])
    #print("%d interdistances computed" % (len(interdistances)))
    return interdistances
    


class Scanner:
    def __init__(s, name, orientation):
        s.name = name
        s.beacons =  None
        s.local2world = orientation
        s.world_offset = array([ 0, 0, 0], dtype=int32)
        s.interdistances = None
    
    def print(s):
        print(s.name)
        print(s.beacons)
        
    def getWorldBeacons(s):
        n,M = s.local2world
        r = s.beacons
        r = s.beacons.dot(M)
        r = r + s.world_offset        
        return r
        
# make list of scanners from text
def parse(text):
    scanners=[]
    for scanner in text.split("\n\n"):
        lines = scanner.split("\n")
        n = lines[0][4:].split(" ---")[0]
        s = Scanner(n, O[0])
        for p in lines[1:]:
            point = [int(x) for x in p.split(",")]
            point = array(point, dtype=int32)
            if s.beacons is None:
                s.beacons = point
            else:
                s.beacons = vstack( [s.beacons, point])
                
        s.interdistances = computeDistances(s.beacons)
        scanners.append(s)
    print("Read %d scanners"%(len(scanners)))
    
    return scanners

def parse_fn(fn):
    print("----parsing ", fn ,"----")
    return parse(open(fn).read())

def findcommon(A,B):
    r = 0
    for a in A:
        if a in B:
            r += 1
    return r



def assemble(scanners):
    mapped_points = None
    mapped_interdistances = None
    pool = deepcopy(scanners)
    
    #iterate over all scanners
    running = len(pool)
    while len(pool):
        #backoff if no result
        if running <= 0:
            #No match
            return None                
        running -= 1
        lmp = 0
        
        if not mapped_interdistances is None:
            #sort pool
            for s in pool:
                s.distances_matched = findcommon(s.interdistances, mapped_interdistances)
            pool.sort(key=lambda x: x.distances_matched)
            
            pool = list(reversed(pool))
            #for s in pool:
            #    print(s.distances_matched)
            
        
        if not mapped_points is None:
            lmp = len(mapped_points)
        print("%s, there are %d unmapped scanners -  (mapped: %d)" % (pool[0].name, len(pool), lmp))
        
        
        s=pool.pop(0)
        
        
        
        
        #First scanner is used as origin
        if mapped_points is None:
            print(s.name, "is origin")
            s.local2world = O[0]
            s.world_offset = array([ 0, 0, 0], dtype=int32)    
            mapped_points=s.getWorldBeacons()
            mapped_interdistances = s.interdistances
        else:
            #following scanners are tested against mapped points
            matched = False
            
            #Try each of the 24 orientations
            for o in O:
                #Try scanner with proposed offset and orientation
                s.local2world = o
                s.world_offset = array([0,0,0],dtype=int32)
                pts = s.getWorldBeacons()
                
                offset2mapped = match(pts, mapped_points)                
                if not offset2mapped is None:
                    print("%s's beacons mapped to world points using %s and offset:" % (s.name, n), offset2mapped)
                    
                    #new found beacon resets search lease to rest of beacons
                    running = len(pool)
                    
                    #Apply offset to scanner to align with mapped points
                    s.world_offset = offset2mapped
                    pts = s.getWorldBeacons()
                    
                    
                    if 0:
                        #Verify that they align - remove for speed later
                        verify = match(pts, mapped_points)
                        print(verify)
                        assert(not verify is None and verify[0] == 0 and verify[1] == 0 and verify[2] == 0 )
                    
                    #add new found coordinates
                    mapped_points = concatenate((mapped_points, pts), axis=0)
                    #discard duplicates
                    mapped_points = unique(mapped_points, axis=0)
                    #mapped_interdistances += s.interdistances
                    mapped_interdistances = computeDistances(mapped_points)
                    #print(mapped_points)
                    matched = True
                    #stop trying orientations
                    break
                
            if not matched:
                print(s.name, "no match")
                pool.append(s)                
    print(mapped_points.shape)
    return mapped_points
                
def part1(scanners):
    mapped_points = assemble(scanners)
    if mapped_points is None:
        print("Unable to match")
        return -1
    else:
        num_rows, num_cols = mapped_points.shape
        return num_rows

sp1 = part1(parse_fn("i19_test.txt"))
print("solve 1 test (79)", sp1)
assert(sp1 == 79)

sp1 = part1(parse_fn("i19.txt"))
print("solve 1 test (??)", sp1)


        
    

Spaces: +X+Y+Z +X+Z-Y +X-Y-Z +X-Z+Y -X+Y-Z -X-Z-Y -X-Y+Z -X+Z+Y +Z+Y-X +Z-X-Y +Z-Y+X +Z+X+Y -Z+Y+X -Z+X-Y -Z-Y-X -Z-X+Y +Y-X+Z +Y+Z+X +Y+X-Z +Y-Z-X -Y+X+Z -Y+Z-X -Y-X-Z -Y-Z+X
----parsing  i19_test.txt ----
Read 5 scanners
scanner 0, there are 5 unmapped scanners -  (mapped: 0)
scanner 0 is origin
scanner 1, there are 4 unmapped scanners -  (mapped: 25)
scanner 1's beacons mapped to world points using -Y-Z+X and offset: [   68 -1246   -43]
scanner 3, there are 3 unmapped scanners -  (mapped: 38)
scanner 3's beacons mapped to world points using -Y-Z+X and offset: [  -92 -2380   -20]
scanner 4, there are 2 unmapped scanners -  (mapped: 51)
scanner 4's beacons mapped to world points using -Y-Z+X and offset: [  -20 -1133  1061]
scanner 2, there are 1 unmapped scanners -  (mapped: 65)
scanner 2's beacons mapped to world points using -Y-Z+X and offset: [ 1105 -1205  1229]
(79, 3)
solve 1 test (79) 79
----parsing  i19.txt ----
Read 38 scanners
scanner 0, there are 38 unmapped scanners -  (map

In [ ]:

def getp():
    return randint(-1000,1000)

def generate(num_s):
    seed(42)
    
    scanner_points = []
    
    s = []
    p = []
    common = None
    for i in range(num_s):
        while len(p) < 13:
            p.append((getp(),getp(),getp()))
        common = p[-12:]
        a = []
        a.append("--- scanner %d ---" % (i))
        
        #ox, oy, oz = 0,0,0
        ox, oy, oz = randint(-100,100),randint(-100,100),randint(-100,100)
        print("offset", ox, oy, oz)
        for x,y,z in p:
            a.append("%d,%d,%d" % (x+ox,y+oy,z+oz))                    
        s.append("\n".join(a))
        p = common
    return "\n\n".join(s)

selftest = parse(generate(6))
for st in selftest:
    st.print()

for i in range(len(selftest)-1):
    print(match(selftest[i].getWorldBeacons(), selftest[i+1].getWorldBeacons()))

print(part1(selftest))

    
        

In [ ]:
test = parse_fn("i19_test.txt")
#print(test[0].beacons)

A = test[0].beacons
B = array([[ 404, -588, -901],
 [ 528, -643,  409],
 [-838,  591,  734]
 ], dtype=int32)


nrows, ncols = A.shape
dtype={'names':['f{}'.format(i) for i in range(ncols)],
       'formats':ncols * [A.dtype]}

C = intersect1d(A.view(dtype), B.view(dtype))
print(len(C))


